# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

In [2]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

In [29]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [3]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [5]:
#why is the bracketed number 1000? What does changing it do substantively?
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":819393184,"wgRevisionId":819393184,"wgArticleId":473317,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology articles needing expert attention","Media articles needing expert attention","Articles that may be too long from January 2018","All articles with un

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

In [6]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
(window.RLQ=window.RLQ||[]).push(functio


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [7]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

Now lets open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag.

In [8]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)



Content analysis is a research method for studying documents and communication artifacts, which can be texts of various formats, pictures, audio or video. Social scientists use content analysis to quantify patterns in communication, in a replicable and systematic manner.[1] One of the key advantage of this research method is to analyse social phenomena in a non-invasive way, in contrast to simulating social experiences or collecting survey answers.
Practices and philosophies of content analysis vary between scholarly communities. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful patterns.[2][3] After labeling a large set of media, a researcher is able to statistically estimate the proportions of patterns in the texts, as well as correlations between patterns.


We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [9]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0                                                    
1   \nContent analysis is a research method for st...
2   Practices and philosophies of content analysis...
3   Computers are increasingly used in content ana...
4                                                    
5                                                    
6   Content analysis is best understood as a broad...
7   The simplest and most objective form of conten...
8   A further step in analysis is the distinction ...
9   More generally, content analysis is research u...
10  By having contents of communication available ...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Recently, Arash Heydarian Pashakhanlou has arg...
17  Content analysis can als

Now we have a `DataFrame` containing all relevant text from the page ready to be
processed

If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

In [10]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<_sre.SRE_Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [11]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [12]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

In [30]:
sep_base_url = 'https://plato.stanford.edu/index.html'
sep_exist = 'https://plato.stanford.edu/entries/existentialism/'
exist_save = 'exist_content_analysis.html'


In [31]:
requests.get(sep_exist)

<Response [200]>

In [32]:
sep_ContentRequest = requests.get(sep_exist)
print(sep_ContentRequest.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="ie6 ie"> <![endif]-->
<!--[if IE 7]>    <html class="ie7 ie"> <![endif]-->
<!--[if IE 8]>    <html class="ie8 ie"> <![endif]-->
<!--[if IE 9]>    <html class="ie9 ie"> <![endif]-->
<!--[if !IE]> --> <html> <!-- <![endif]-->
<head>
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>
Existentialism (Stanford Encyclopedia of Philosophy)
</title>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta name="robots" content="noarchive, noodp" />
<meta property="citation_title" content="Existentialism" />
<meta property="citation_author" content="Crowell, Steven" />
<meta property="citation_publication_date" content="2004/08/23" />
<meta name="DC.title" content="Existentialism" />
<meta name="DC.creator" content="Crowell, Steven" />
<meta name="DCTERMS.issued" content="2004-08-23" />
<meta name="DCTERMS.modified" content="2015-03-09" />

<!-- NOTE: Import webfonts using this link: -->
<link href="

In [33]:
sepContentSoup = bs4.BeautifulSoup(sep_ContentRequest.text, 'html.parser')
print(sepContentSoup.text[:2000])






  



Existentialism (Stanford Encyclopedia of Philosophy)



































Stanford Encyclopedia of Philosophy





  Menu 


 Browse

Table of Contents
What's New
Random Entry
Chronological
Archives


 About

Editorial Information
About the SEP
Editorial Board
How to Cite the SEP
Special Characters
Advanced Tools
Contact


 Support SEP

Support the SEP
PDFs for SEP Friends
Make a Donation
SEPIA for Libraries


























  Entry Navigation 


Entry Contents
Bibliography
Academic Tools
Friends PDF Preview 
Author and Citation Info  
Back to Top 












ExistentialismFirst published Mon Aug 23, 2004; substantive revision Mon Mar 9, 2015



Like “rationalism” and “empiricism,”
“existentialism” is a term that belongs to intellectual
history. Its definition is thus to some extent one of historical
convenience. The term was explicitly adopted as a self-description by
Jean-Paul Sartre, and through the wide dissemination of the postwar
literary and ph

In [34]:
exist_save = 'sep_content_analysis.html'

In [35]:
with open(exist_save, mode='w', encoding='utf-8') as f:
    f.write(sep_ContentRequest.text)

In [36]:
#creates variable using BeautifulSoup command which finds all the <p> tags in the text between which are meaningful text
contentPTags = sepContentSoup.body.findAll('p')
#the following for loop will print out the first 5 paragraphs of the relevant text
for pTag in contentPTags[:5]:
    print(pTag.text)
    #the .text eliminates some of the HTML code that would otherwise be displayed by the 'print' command



Like “rationalism” and “empiricism,”
“existentialism” is a term that belongs to intellectual
history. Its definition is thus to some extent one of historical
convenience. The term was explicitly adopted as a self-description by
Jean-Paul Sartre, and through the wide dissemination of the postwar
literary and philosophical output of Sartre and his
associates—notably Simone de Beauvoir, Maurice Merleau-Ponty,
and Albert Camus—existentialism became identified with a
cultural movement that flourished in Europe in the 1940s and
1950s. Among the major philosophers identified as existentialists
(many of whom—for instance Camus and Heidegger—repudiated
the label) were Karl Jaspers, Martin Heidegger, and Martin Buber in
Germany, Jean Wahl and Gabriel Marcel in France, the Spaniards
José Ortega y Gasset and Miguel de Unamuno, and the Russians
Nikolai Berdyaev and Lev Shestov. The nineteenth century philosophers,
Søren Kierkegaard and Friedrich Nietzsche, came to be seen as
precursors of the mov

In [37]:
#creates an empty list 'contetParagraphs' then 'appends' it with the previously created contentPTags varaible, adding the
#paragraphs that are indiccated by the <p> tag
contentParagraphs = []
for pTag in contentPTags:
    contentParagraphs.append(pTag.text)

In [38]:
#creates a data
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0   \n\nLike “rationalism” and “empiricism,”\n“exi...
1   \n\nIt is sometimes suggested, therefore, that...
2   \n\nOn the existential view, to understand wha...
3   \n\n“Existentialism”, therefore, may be define...
4   \n\nSartre's existentialism drew its immediate...
5   \n\nAs Sartre and Merleau-Ponty would later do...
6   \n\nA phenomenology of consciousness, then, ex...
7   \n\nKierkegaard developed this problem in the ...
8   \n\nIn Kierkegaard, the singularity of existen...
9   \n\nTo solve this problem there must be a norm...
10  \n\nIn contrast to the singularity of this mov...
11  \n\nFor Kierkegaard existence emerges as a phi...
12  \n\nResponding in part to the cultural situati...
13  \n\nYet this is not the end of the story for N...
14  \n\nThus, for Nietzsche, existence emerges as ...
15  \n\nAs did Kierkegaard, then, Nietzsche uncove...
16  \n\nSartre's slogan—“existence precedes\nessen...
17  \n\nAt first, it seems h

In [60]:
find_sartre = re.compile(r'Sartre', re.IGNORECASE)

for i in range(0,len(contentParagraphsDF)):
    print(find_sartre.findall(contentParagraphsDF.iloc[i,0]))


['Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre']
[]
[]
['Sartre']
['Sartre']
[]
['Sartre']
[]
[]
[]
[]
[]
[]
[]
[]
['Sartre']
[]
['Sartre', 'Sartre']
[]
[]
[]
[]
['Sartre']
[]
['Sartre', 'Sartre']
[]
[]
[]
[]
[]
[]
[]
[]
['Sartre']
['Sartre']
['Sartre', 'Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre', 'Sartre']
['Sartre']
['Sartre']
['Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre', 'Sartre', 'Sartre', 'Sartre', 'Sartre']
['Sartre']
['Sartre', 'Sartre', 'Sartre']
[]
[]
['Sartre', 'Sartre']
[]
[]
[]
[]
['Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre']
['Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre', 'Sartre', 'Sartre']
['Sartre', 'Sartre', 'Sartre', 'Sartre']
['Sartre']
['Sartre']
['Sartre']
['Sartre']
[]
[]
[]
['Sartre']
[]
[]
[]
[]
[]


In [59]:
find_cite = re.compile(r'[A-Z][a-z]+..\d{4}')

for i in range(0,len(contentParagraphsDF)):
    print(find_cite.findall(contentParagraphsDF.iloc[i,0]))

[]
[]
[]
['Kaufmann 1968']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Fackenheim 1961']
['Heidegger 1985']
[]
['Taylor 1985']
[]
[]
[]
[]
[]
['Sartre (1992']
[]
[]
[]
[]
['Nehamas 1998', 'Ricoeur 1992']
[]
[]
[]
[]
[]
[]
['Sartre (1992']
[]
[]
['Apel (1973', 'Sartre 1992']
['Sartre 1992', 'Sartre 1992']
['Sartre 1992']
['Sartre 1992']
[]
[]
[]
[]
['Heidegger 1962', 'Heidegger 1962']
['Heidegger 1962']
[]
[]
[]
[]
['Sartre 1968']
[]
['Sartre 1968']
[]
['Bernasconi 2003', 'Ratcliffe (2008']
['Carr (1986', 'Dreyfus (1979', 'Haugeland (1998', 'Korsgaard (1996', 'Moran (2001']
['Cooper (1999', 'Schrift (1995', 'Reynolds (2006', 'Reynolds (2004']
['Baring (2011', 'Bernasconi (2012', 'Simons (2013', 'Woodward 2011', 'Crowell 2012', 'Aho (2014']
[]
[]
[]
[]
[]
[]
[]
[]


In [58]:
find_citations = re.compile(r'[A-Z][a-z]+..\(\d{4}\)')

for i in range(0,len(contentParagraphsDF)):
    print(find_citations.findall(contentParagraphsDF.iloc[i,0]))

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Ratcliffe (2008)']
['Carr (1986)', 'Dreyfus (1979)', 'Haugeland (1998)', 'Korsgaard (1996)', 'Moran (2001)']
['Cooper (1999)', 'Schrift (1995)', 'Reynolds (2006)', 'Reynolds (2004)']
['Baring (2011)', 'Bernasconi (2012)', 'Simons (2013)', 'Aho (2014)']
[]
[]
[]
[]
[]
[]
[]
[]


In [57]:
find_names = re.compile(r'[A-Z][a-z]+..[A-Z][a-z]+')

for i in range(0,len(contentParagraphsDF)):
    print(find_names.findall(contentParagraphsDF.iloc[i,0]))
    
    #commands is meant to find names, first and last, but the form is too brittle, it can't capture hyphenated names like 
    #Jean-Paul Sartre, as well as picking up a bunch of junk that aren't names such as two proper nouns.

['Jean-Paul', 'Beauvoir, Maurice', 'Merleau-Ponty', 'Albert Camus', 'Karl Jaspers', 'Martin Heidegger', 'Martin Buber', 'Germany, Jean', 'Gabriel Marcel', 'José Ortega', 'Nikolai Berdyaev', 'Lev Shestov', 'Friedrich Nietzsche', 'Dialectical Reason', 'Dostoevsky, Ibsen', 'Jean Genet', 'André Gide', 'André Malraux', 'Samuel Beckett', 'Norwegian Knut', 'Eugene Ionesco', 'Alberto Giacometti', 'Abstract Expressionists', 'Jackson Pollock', 'Jean-Luc', 'Ingmar Bergman', 'Woody Allen']
['Rudolf Bultmann', 'Paul Tillich', 'Karl Barth', 'Ludwig Binswanger', 'Medard Boss', 'Viktor Frankl']
[]
[]
['Martin Heidegger']
['Merleau-Ponty', 'Edmund Husserl']
['Friedrich Nietzsche', 'Augustine, Pascal']
['God. Subsequent', 'Heidegger, Jaspers']
[]
['Concluding Unscientific', 'Religion Within', 'Reason Alone', 'God. His']
[]
['For Kierkegaard']
['Fyodor Dostoevsky', 'Brothers Karamazov', 'Christian God', 'Judeo-Christian', 'Thus Nietzsche']
['Kierkegaard. If']
['The Gay', 'For Nietzsche']
[]
[]
[]
[]
['Ch

In [64]:
find_phenom = re.compile(r'phenomenolo', re.IGNORECASE)

for i in range(0,len(contentParagraphsDF)):
    print(find_phenom.findall(contentParagraphsDF.iloc[i,0]))

[]
[]
[]
[]
[]
['phenomenolo', 'phenomenolo', 'phenomenolo']
['phenomenolo', 'phenomenolo', 'phenomenolo']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['phenomenolo', 'phenomenolo']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Phenomenolo']
[]
[]
[]
[]
[]
['phenomenolo']
[]
[]
[]
[]
[]



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [60]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('^/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://plato.stanford.edu/arendt/', 64, 'Arendt, Hannah'), ('https://plato.stanford.edu/beauvoir/', 64, 'Beauvoir, Simone de'), ('https://plato.stanford.edu/buber/', 64, 'Buber, Martin'), ('https://plato.stanford.edu/heidegger/', 64, 'Heidegger, Martin'), ('https://plato.stanford.edu/hermeneutics/', 64, 'hermeneutics'), ('https://plato.stanford.edu/husserl/', 64, 'Husserl, Edmund'), ('https://plato.stanford.edu/jaspers/', 64, 'Jaspers, Karl'), ('https://plato.stanford.edu/kierkegaard/', 64, 'Kierkegaard, Søren'), ('https://plato.stanford.edu/levinas/', 64, 'Levinas, Emmanuel'), ('https://plato.stanford.edu/marcel/', 64, 'Marcel, Gabriel (-Honoré)')]


We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [16]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,,https://en.wikipedia.org/wiki/Content_analysis,0
1,\nContent analysis is a research method for st...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,3
4,,https://en.wikipedia.org/wiki/Content_analysis,4
5,,https://en.wikipedia.org/wiki/Content_analysis,5
6,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,6
7,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,7
8,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,8
9,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [17]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [18]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,,https://en.wikipedia.org/wiki/Content_analysis,None,None
1,1,\nContent analysis is a research method for st...,https://en.wikipedia.org/wiki/Content_analysis,None,None
2,2,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,None,None
3,3,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,None,None
4,4,,https://en.wikipedia.org/wiki/Content_analysis,None,None
5,5,,https://en.wikipedia.org/wiki/Content_analysis,None,None
6,6,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,None,None
7,7,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,None,None
8,8,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,None,None
9,9,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



In [71]:
sep_base_url = 'https://plato.stanford.edu/entries'

In [87]:
#wikipedia_base_url = 'https://en.wikipedia.org'
sep_base_url = 'https://plato.stanford.edu/entries'
otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('\.\.'))
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(sep_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

#it looks at first as though the command was succesful and that it found all the relevant URLs but upon closer inspection
#one can notice that there is a part of each URL missing, between the .edu/ and the philosophers name, there should be a 
#/entries but for whatever reason the command is not pulling this part of the URL from the page. I tried fixing it but
#can't find where it is going wrong though I suspect it is in the command preceding these comments. This means that the 
#DataFrame below is incomplete as it only includes the content of'404 Page not Found" pages. 

[('https://plato.stanford.edu/arendt/', 64, 'Arendt, Hannah'), ('https://plato.stanford.edu/beauvoir/', 64, 'Beauvoir, Simone de'), ('https://plato.stanford.edu/buber/', 64, 'Buber, Martin'), ('https://plato.stanford.edu/heidegger/', 64, 'Heidegger, Martin'), ('https://plato.stanford.edu/hermeneutics/', 64, 'hermeneutics'), ('https://plato.stanford.edu/husserl/', 64, 'Husserl, Edmund'), ('https://plato.stanford.edu/jaspers/', 64, 'Jaspers, Karl'), ('https://plato.stanford.edu/kierkegaard/', 64, 'Kierkegaard, Søren'), ('https://plato.stanford.edu/levinas/', 64, 'Levinas, Emmanuel'), ('https://plato.stanford.edu/marcel/', 64, 'Marcel, Gabriel (-Honoré)')]


In [88]:
contentParagraphsDF['source'] = [sep_exist] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,"\n\nLike “rationalism” and “empiricism,”\n“exi...",https://plato.stanford.edu/entries/existential...,None,None
1,1,"\n\nIt is sometimes suggested, therefore, that...",https://plato.stanford.edu/entries/existential...,None,None
2,2,"\n\nOn the existential view, to understand wha...",https://plato.stanford.edu/entries/existential...,None,None
3,3,"\n\n“Existentialism”, therefore, may be define...",https://plato.stanford.edu/entries/existential...,None,None
4,4,\n\nSartre's existentialism drew its immediate...,https://plato.stanford.edu/entries/existential...,None,None
5,5,\n\nAs Sartre and Merleau-Ponty would later do...,https://plato.stanford.edu/entries/existential...,None,None
6,6,"\n\nA phenomenology of consciousness, then, ex...",https://plato.stanford.edu/entries/existential...,None,None
7,7,\n\nKierkegaard developed this problem in the ...,https://plato.stanford.edu/entries/existential...,None,None
8,8,"\n\nIn Kierkegaard, the singularity of existen...",https://plato.stanford.edu/entries/existential...,None,None
9,9,\n\nTo solve this problem there must be a norm...,https://plato.stanford.edu/entries/existential...,None,None


In [89]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromsepPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(pTag.text)
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

In [90]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromsepPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-number,paragraph-text,source,source-paragraph-number,source-paragraph-text
0,0,"\n\nLike “rationalism” and “empiricism,”\n“exi...",https://plato.stanford.edu/entries/existential...,None,None
1,1,"\n\nIt is sometimes suggested, therefore, that...",https://plato.stanford.edu/entries/existential...,None,None
2,2,"\n\nOn the existential view, to understand wha...",https://plato.stanford.edu/entries/existential...,None,None
3,3,"\n\n“Existentialism”, therefore, may be define...",https://plato.stanford.edu/entries/existential...,None,None
4,4,\n\nSartre's existentialism drew its immediate...,https://plato.stanford.edu/entries/existential...,None,None
5,5,\n\nAs Sartre and Merleau-Ponty would later do...,https://plato.stanford.edu/entries/existential...,None,None
6,6,"\n\nA phenomenology of consciousness, then, ex...",https://plato.stanford.edu/entries/existential...,None,None
7,7,\n\nKierkegaard developed this problem in the ...,https://plato.stanford.edu/entries/existential...,None,None
8,8,"\n\nIn Kierkegaard, the singularity of existen...",https://plato.stanford.edu/entries/existential...,None,None
9,9,\n\nTo solve this problem there must be a norm...,https://plato.stanford.edu/entries/existential...,None,None


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [65]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":2964,"posts-type":false,"posts":[{"id":"169306612050","url":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/169306612050","url-with-slug":"http:\/\/lolcats-lol-cat.tumblr.com\/post\/169306612050\/kitty-mind-blown","type":"photo","date-gmt":"2018-01-04 15:00:13 GMT","date":"Thu, 04 Jan 2018 16:00:13","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1515078013,"format":"html","reblog-key":"TZzKhxon","slug":"kitty-mind-blown","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"169306612050\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_169306612050\"><iframe id=\"like_iframe_169306612050\" src=\"http:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=fc298e85f978b8662a643fe0a6b8c638#name=lolcats-lol-cat&amp;post_id=169306612050&amp;co

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [66]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [ ]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [68]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,date,id,photo-type,photo-url,tags
0,"Thu, 04 Jan 2018 16:00:13",169306612050,jpg,http://78.media.tumblr.com/704a681944a94a1fac1...,"[cat, cats, lol, lolcat, lolcats]"
1,"Thu, 04 Jan 2018 14:00:41",169303827051,png,http://78.media.tumblr.com/1ec2cc33b059d657351...,"[cat, cats, lol, lolcat, lolcats]"
2,"Thu, 04 Jan 2018 12:00:34",169301536211,png,http://78.media.tumblr.com/80dc7c42c35a2dbce58...,"[cat, cats, lol, lolcat, lolcats]"
3,"Tue, 02 Jan 2018 04:00:09",169209300706,jpg,http://78.media.tumblr.com/94b8dbe91683bd01551...,"[cat, cats, lol, lolcat, lolcats]"
4,"Tue, 02 Jan 2018 02:00:42",169205268941,jpg,http://78.media.tumblr.com/e3bbb26992ac10e0234...,"[cat, cats, lol, lolcat, lolcats]"
5,"Sun, 31 Dec 2017 08:00:10",169141076975,png,http://78.media.tumblr.com/5ef1ee635831085377d...,"[cat, cats, lol, lolcat, lolcats]"
6,"Sun, 31 Dec 2017 06:00:35",169137806653,jpg,http://78.media.tumblr.com/9dc91b972231d1e6250...,"[cat, cats, lol, lolcat, lolcats]"
7,"Sun, 31 Dec 2017 04:00:38",169134323015,png,http://78.media.tumblr.com/f62a86855bb0ed8bf1e...,"[cat, cats, lol, lolcat, lolcats]"
8,"Sun, 31 Dec 2017 02:00:42",169130694997,jpg,http://78.media.tumblr.com/760cd9b9017708c90d5...,"[cat, cats, lol, lolcat, lolcats]"
9,"Sun, 24 Dec 2017 08:00:08",168883132580,jpg,http://78.media.tumblr.com/c078fc21432061a41ec...,"[cat, cats, lol, lolcat, lolcats]"


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [69]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [70]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. LEts tart by looking at the Shakespeare files in the `data` directory 

Then to load all the files in `../data/Shakespeare` we can use a for loop with `scandir`:

In [72]:
targetDir = '../data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [73]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
rape_of_lucrece.txt,THE RAPE OF LUCRECE\n\nby William Shakespeare\...
tempest.txt,"The Tempest\n\nActus primus, Scena prima.\n\nA..."
midsummer_nights_dream.txt,A MIDSUMMER NIGHT'S DREAM\n\nby William Shakes...
king_richard_3.txt,KING RICHARD III\n\nby William Shakespeare\n\n...
anthonie_and_cleopatra.txt,"The Tragedie of Anthonie, and Cleopatra\n\nAct..."
king_henry_6_p1.txt,Dramatis Personae\n\nKING HENRY the Sixth\nDUK...
loves_labors_lost.txt,LOVE'S LABOUR'S LOST\n\nby William Shakespeare...
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
pericles_prince_of_tyre.txt,PERICLES PRINCE OF TYRE\n\nby William Shakespe...
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [94]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [75]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [76]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [77]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [78]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [79]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [80]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [81]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


In [3]:
with open('../Joseph Conrad/Amy Foster.txt') as f:
    heartofdarkness = f.read()
    print(heartofdarkness[-700:])


ject
Gutenberg-tm eBooks with only a loose network of volunteer support.


Project Gutenberg-tm eBooks are often created from several printed
editions, all of which are confirmed as Public Domain in the U.S.
unless a copyright notice is included.  Thus, we do not necessarily
keep eBooks in compliance with any particular paper edition.


Most people start at our Web site which has the main PG search facility:

     http://www.gutenberg.org

This Web site includes information about Project Gutenberg-tm,
including how to make donations to the Project Gutenberg Literary
Archive Foundation, how to help produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.



In [4]:
targetDir = '../Joseph Conrad'
conradText = []
conradFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path) as f:
        conradText.append(f.read())
    conradFileName.append(file.name)

In [5]:
conrad_df = pandas.DataFrame({'text' : conradText}, index = conradFileName)
conrad_df

,text
Amy Foster.txt,"﻿The Project Gutenberg EBook of Amy Foster, by..."
The Shadow-Line.txt,﻿The Project Gutenberg EBook of The Shadow-Lin...
Youth.txt,"﻿The Project Gutenberg EBook of Youth, by Jose..."
Heart_of_Darkness.txt,﻿The Project Gutenberg EBook of Heart of Darkn...
An Outcast of the Islands.txt,﻿The Project Gutenberg EBook of An Outcast of ...
Victory.txt,"﻿The Project Gutenberg EBook of Victory, by Jo..."
Under Western Eyes.txt,﻿The Project Gutenberg EBook of Under Western ...
Tales of Unrest.txt,﻿The Project Gutenberg EBook of Tales of Unres...
Almayer's Folly.txt,"﻿The Project Gutenberg eBook, Almayer's Folly,..."
Nostromo.txt,﻿Project Gutenberg’s Nostromo: A Tale of the S...


In [16]:
find_horror = re.compile(r'the horror.{2}', re.IGNORECASE)

for i in range(0,len(conrad_df)):
    print(find_horror.findall(conrad_df.iloc[i,0]))

[]
[]
[]
['The horror! ', "The horror!'", "The horror!'", 'The horror! ', "The horror!'", 'The horror! ', "The horror!'"]
['the horror o', 'the horror o', 'the horror o']
['the horrors ', 'the horrors,', 'the horrors ']
['the horrors ', 'the horrors ', 'the horrors ']
['the horror o']
['the horror o']
[]
[]
[]
[]


In [17]:
find_kurtz = re.compile(r'Kurtz', re.IGNORECASE)

for i in range(0,len(conrad_df)):
    print(find_kurtz.findall(conrad_df.iloc[i,0]))

[]
[]
[]
['Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 'Kurtz', 

In [22]:
find_conrad = re.compile(r'darkness', re.IGNORECASE)

for i in range(0,len(conrad_df)):
    print(find_conrad.findall(conrad_df.iloc[i,0]))

['darkness', 'darkness', 'darkness', 'darkness', 'darkness']
['darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness']
['darkness', 'darkness']
['Darkness', 'Darkness', 'DARKNESS', 'DARKNESS', 'darkness', 'darkness', 'darkness', 'darkness', 'Darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'Darkness', 'DARKNESS']
['darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness', 'darkness',

In [27]:
find_conrad = re.compile(r'boat', re.IGNORECASE)

for i in range(0,len(conrad_df)):
    print(find_conrad.findall(conrad_df.iloc[i,0]))

[]
['boat', 'boat', 'boat', 'boat', 'boat']
['boat', 'boat', 'boat', 'boat', 'Boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat']
['boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat']
['boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'Boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', 'boat', '

In [28]:
find_conrad = re.compile(r'ship', re.IGNORECASE)

for i in range(0,len(conrad_df)):
    print(find_conrad.findall(conrad_df.iloc[i,0]))

['ship', 'Ship', 'ship', 'Ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'Ship', 'ship', 'ship', 'ship', 'ship', 'ship']
['ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'Ship', 'Ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'Ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'Ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship', 'ship'

In [29]:
find_conrad = re.compile(r'[A-Z][a-z]+\s[A-Z][a-z]+..')

for i in range(0,len(conrad_df)):
    print(find_conrad.findall(conrad_df.iloc[i,0]))

['The Project G', 'Amy Foster, ', 'Joseph Conrad', 'Project Gutenberg L', 'Amy Foster', 'Joseph Conrad', 'Release Date: ', 'Last Updated: ', 'Judith Boss a', 'David Widger', 'By Joseph C', 'High Street a', 'Ship Inn” ', 'Colebrook\nChurch. ', 'Isaac Foster, ', 'New Barns F', 'For Mrs. ', 'New Barns i', 'Central Europe b', 'High\nStreet, ', 'Talfourd Hill, ', 'Holy Image i', 'Carmelite Convent d', 'American Kaiser w', 'The American K', 'West\nColebrook h', 'New Barns F', 'Amy Foster’s', 'Darnford Market) ', 'Amy Foster s', 'Herzogin Sophia-D', 'Emigration Agencies’ ', 'Brenzett Coastguard s', 'The\nHamburg s', 'Ship Inn,’', 'Brenzett Church.', 'West Colebrook, ', 'Amy Foster a', 'Amy\nFoster, ', 'But Smith w', 'When Mr. ', 'New Barns,’', 'Miss Swaffer, ', 'Miss Swaffer, ', 'Baptist Chapel)-', 'Miss Swaffer’s', 'Town Clerk o', 'Old Swaffer w', 'Miss\nSwaffer, ', 'Amy\nFoster’s', 'Yanko\nGoorall--', 'Amy Foster a', 'The Fosters, ', 'Miss Swaffer', 'Miss Swaffer’s', 'Miss Swaffer w', 'Bert